<a href="https://colab.research.google.com/github/sayed-Gomaa/AlexNet_paper-implementation/blob/main/Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Configuration environment
import os

os.environ['KAGGLE_USERNAME'] = "sayedgomaa" # username from the json file
os.environ['KAGGLE_KEY'] = "c310dd811930170fe3f071f577c08b54" # key from the json file

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/Paper')

! pwd

In [ ]:
!kaggle datasets download -d puneet6060/intel-image-classification

In [ ]:
! pwd

In [ ]:
! unzip intel-image-classification.zip

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import os
import shutil
import glob

In [ ]:

def plot_imgs(item_dir, top=10):
    all_item_dirs = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dirs][100:106]
  
    plt.figure(figsize=(10, 10))
  
    for idx, img_path in enumerate(item_files):
        plt.subplot(1, 6, idx+1)        
        img = plt.imread(img_path)
        plt.tight_layout() 
        plt.axis('off')
        plt.imshow(img, cmap='gray')
        

In [ ]:
! ls

In [ ]:
training_data_path="seg_train/seg_train"
testing_data_path="seg_test/seg_test"

In [ ]:
img_height = 227
img_width = 227

training_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=40,
                                                                    shear_range=0.2,
                                                                    zoom_range=0.2,
                                                                    horizontal_flip=True,
                                                                    vertical_flip=True,
                                                                    rescale=1/255.0,
                                                                    validation_split=0.25)

train_generator = training_data_gen.flow_from_directory(training_data_path,
                                                        target_size=(img_height, img_width),
                                                        batch_size=64,
                                                        shuffle=True,
                                                        class_mode='categorical',
                                                        subset='training') # set as training data

validation_generator = training_data_gen.flow_from_directory(training_data_path, # same directory as training data
                                                             target_size=(img_height, img_width),
                                                             batch_size=64,
                                                             shuffle=True,
                                                             class_mode='categorical',
                                                             subset='validation') # set as validation data
 

testing_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

test_generator = testing_data_gen.flow_from_directory(testing_data_path,
                                                      target_size=(img_height, img_width),
                                                      batch_size=60,
                                                      seed=0,
                                                      shuffle=False,
                                                      class_mode='categorical') 

In [ ]:
 img,lebel=next(train_generator)

In [ ]:
%matplotlib inline
def plotImages(images_arr):
    fig, axes = plt.subplots(2, 4, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(img)

In [ ]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=96,kernel_size=(11,11),strides=4,activation='relu',padding='same',input_shape=(227,227,3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.ZeroPadding2D(padding=(2, 2)))
model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=(5,5),activation='relu',padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2))
 

model.add(tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(filters=384,kernel_size=(3,3),activation='relu',padding='same'))

model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2))


model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(units=4096,activation='relu'))
model.add(tf.keras.layers.Dropout(.5))


model.add(tf.keras.layers.Dense(units=4096,activation='relu'))
model.add(tf.keras.layers.Dropout(.5))

model.add(tf.keras.layers.Dense(units=6,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0005),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,validation_data=validation_generator, epochs=20,verbose=1)

In [ ]:
model.save('model.h5')

In [ ]:
predect = model.predict(test_generator,batch_size=10,verbose=1)

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(["Training Accuracy","Validation Accuracy"])
plt.show()


#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.title('Training and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(["Training Loss","Validation Loss"])
plt.show()

# Desired output. Charts with training and validation metrics. 
